# 测试集图像分类预测结果

使用训练好的图像分类模型，预测测试集的所有图像，得到预测结果表格。

## 导入工具包

In [1]:
import os
from tqdm import tqdm

import numpy as np
import pandas as pd

from PIL import Image

import torch
import torch.nn.functional as F

# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 图像预处理

In [2]:
from torchvision import transforms
from COME15KClassDataset import set_data_loader
from torchvision import datasets

# # 训练集图像预处理：缩放裁剪、图像增强、转 Tensor、归一化
# train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
#                                       transforms.RandomHorizontalFlip(),
#                                       transforms.ToTensor(),
#                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#                                      ])

# 测试集图像预处理-RCTN：缩放、裁剪、转 Tensor、归一化
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

/home/mark/.local/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


## 载入测试集（和训练代码教程相同）

In [5]:
# 数据集文件夹路径
dataset_name = 'test_easy_classes'
model_path = '2023-09-26-00:48_max_epoch_50/'
model_name = 'retrain_COME15K_checkpoint-best-easy-0.757-Medium.pth.tar'
dataset_dir = '../data_class_txt/'+ dataset_name + '.txt'
# 类别名称 和 ID索引号 的映射字典
class_names_dic = {0: 'covering', 1: 'device', 2: 'domestic_animal', 3: 'mater', 4: 'person', 5: 'plant',
                       6: 'structure', 7: 'vertebrate'}

test_dataset_loader_easy = set_data_loader(dataset_attr_word="test_easy", batch_size=1, size=512, shuffle=False,
                                           transforms_compose=test_transform, dataset_dir='data/SOD-SemanticDataset')
# 载入测试集
print('测试集图像数量', len(test_dataset_loader_easy))
print('类别个数', len(class_names_dic))
print('各类别名称', list(class_names_dic.values()))
# 获得类别名称
classes = list(class_names_dic.values())
print(classes)

测试集图像数量 4600
类别个数 8
各类别名称 ['covering', 'device', 'domestic_animal', 'mater', 'person', 'plant', 'structure', 'vertebrate']
['covering', 'device', 'domestic_animal', 'mater', 'person', 'plant', 'structure', 'vertebrate']


## 导入训练好的模型(导入参数)

In [8]:
# from network import ShuffleNetV2_Plus
# # init model
# architecture = [0, 0, 3, 1, 1, 1, 0, 0, 2, 0, 2, 1, 1, 0, 2, 0, 2, 1, 3, 2]
# model = ShuffleNetV2_Plus(architecture=architecture, n_class=class_names_dic.__len__(), model_size="Medium")
# weight_path = model_path + model_name
# trained_weight = torch.load(weight_path)
# model.load_state_dict(trained_weight['state_dict'], strict=True)
# model = model.eval().to(device)

## 导入训练好的模型(导入模型 + 参数)

In [9]:
model_and_weight_path = 'models/' + model_path + model_name
model = torch.load(model_and_weight_path)
model = model.eval().to(device)

## 表格A-测试集图像路径及标注

In [115]:
data_path = test_dataset_loader_easy.dataset.data_path

In [116]:
img_paths = test_dataset_loader_easy.dataset.images
img_lables = test_dataset_loader_easy.dataset.labels

In [117]:
df = pd.DataFrame()
df['图像路径'] = [data_path + img_name for img_name in img_paths]
df['标注类别ID'] = img_lables
df['标注类别名称'] = [class_names_dic.get(ID) for ID in img_lables]

In [118]:
df

,图像路径,标注类别ID,标注类别名称
0,data/SOD-SemanticDataset/test/COME15K-Easy/COM...,0,covering
1,data/SOD-SemanticDataset/test/COME15K-Easy/COM...,0,covering
2,data/SOD-SemanticDataset/test/COME15K-Easy/COM...,0,covering
3,data/SOD-SemanticDataset/test/COME15K-Easy/COM...,0,covering
4,data/SOD-SemanticDataset/test/COME15K-Easy/COM...,0,covering
...,...,...,...
4595,data/SOD-SemanticDataset/test/COME15K-Easy/COM...,7,vertebrate
4596,data/SOD-SemanticDataset/test/COME15K-Easy/COM...,7,vertebrate
4597,data/SOD-SemanticDataset/test/COME15K-Easy/COM...,7,vertebrate
4598,data/SOD-SemanticDataset/test/COME15K-Easy/COM...,7,vertebrate


## 表格B-测试集每张图像的图像分类预测结果，以及各类别置信度

In [119]:
# 记录 top-n 预测结果
n = 3

In [120]:
df_pred = pd.DataFrame()
for idx, row in tqdm(df.iterrows()):
    img_path = row['图像路径']
    img_pil = Image.open(img_path).convert('RGB')
    input_img = test_transform(img_pil).unsqueeze(0).to(device) # 预处理
    pred_logits = model(input_img) # 执行前向预测，得到所有类别的 logit 预测分数
    pred_softmax = F.softmax(pred_logits, dim=1) # 对 logit 分数做 softmax 运算

    pred_dict = {}

    top_n = torch.topk(pred_softmax, n) # 取置信度最大的 n 个结果
    pred_ids = top_n[1].cpu().detach().numpy().squeeze() # 解析出类别
    
    # top-n 预测结果
    for i in range(1, n+1):
        pred_dict['top-{}-预测ID'.format(i)] = pred_ids[i-1]
        pred_dict['top-{}-预测名称'.format(i)] = class_names_dic.get(pred_ids[i-1])
    pred_dict['top-n预测正确'] = row['标注类别ID'] in pred_ids
    # 每个类别的预测置信度
    for idx, each in enumerate(classes):
        pred_dict['{}-预测置信度'.format(each)] = pred_softmax[0][idx].cpu().detach().numpy()
        
    df_pred = df_pred._append(pred_dict, ignore_index=True)

4600it [00:54, 83.76it/s] 


In [121]:
df_pred

,top-1-预测ID,top-1-预测名称,top-2-预测ID,top-2-预测名称,top-3-预测ID,top-3-预测名称,top-n预测正确,covering-预测置信度,device-预测置信度,domestic_animal-预测置信度,mater-预测置信度,person-预测置信度,plant-预测置信度,structure-预测置信度,vertebrate-预测置信度
0,3,mater,1,device,6,structure,False,0.0534828,0.22173585,0.0018525249,0.47864333,0.015742665,0.0074155363,0.20646054,0.014666786
1,2,domestic_animal,4,person,7,vertebrate,False,0.06581023,0.0012001912,0.7308288,0.0071285344,0.097066306,0.0016192131,0.0026739398,0.093672715
2,4,person,5,plant,6,structure,False,0.09953721,0.01564171,0.0029690883,0.060892075,0.29755738,0.25626463,0.15151578,0.11562208
3,0,covering,4,person,3,mater,True,0.51454276,0.0022692764,0.005742688,0.012136732,0.44652367,0.0052267183,0.008115943,0.005442206
4,4,person,0,covering,7,vertebrate,True,0.30038536,0.001374066,0.025110465,0.004682911,0.6179109,0.0060193,0.003014118,0.041502878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,4,person,0,covering,3,mater,False,0.11008805,0.0054829586,0.0037603693,0.01637851,0.85486096,0.0021986892,0.003526009,0.0037044452
4596,7,vertebrate,6,structure,2,domestic_animal,True,0.025481148,0.030964468,0.15987156,0.12275824,0.046617243,0.005959204,0.2390001,0.36934802
4597,7,vertebrate,3,mater,1,device,True,0.000114533395,0.00047907434,0.00039487163,0.017899925,0.0002784974,0.00034024936,0.0002899217,0.9802029
4598,7,vertebrate,2,domestic_animal,3,mater,True,0.0053928727,0.0034533,0.04464799,0.024935547,0.009527823,0.0040310402,0.008071735,0.8999396


## 拼接AB两张表格

In [122]:
df = pd.concat([df, df_pred], axis=1)

In [123]:
df

,图像路径,标注类别ID,标注类别名称,top-1-预测ID,top-1-预测名称,top-2-预测ID,top-2-预测名称,top-3-预测ID,top-3-预测名称,top-n预测正确,covering-预测置信度,device-预测置信度,domestic_animal-预测置信度,mater-预测置信度,person-预测置信度,plant-预测置信度,structure-预测置信度,vertebrate-预测置信度
0,data/SOD-SemanticDataset/test/COME15K-Easy/COM...,0,covering,3,mater,1,device,6,structure,False,0.0534828,0.22173585,0.0018525249,0.47864333,0.015742665,0.0074155363,0.20646054,0.014666786
1,data/SOD-SemanticDataset/test/COME15K-Easy/COM...,0,covering,2,domestic_animal,4,person,7,vertebrate,False,0.06581023,0.0012001912,0.7308288,0.0071285344,0.097066306,0.0016192131,0.0026739398,0.093672715
2,data/SOD-SemanticDataset/test/COME15K-Easy/COM...,0,covering,4,person,5,plant,6,structure,False,0.09953721,0.01564171,0.0029690883,0.060892075,0.29755738,0.25626463,0.15151578,0.11562208
3,data/SOD-SemanticDataset/test/COME15K-Easy/COM...,0,covering,0,covering,4,person,3,mater,True,0.51454276,0.0022692764,0.005742688,0.012136732,0.44652367,0.0052267183,0.008115943,0.005442206
4,data/SOD-SemanticDataset/test/COME15K-Easy/COM...,0,covering,4,person,0,covering,7,vertebrate,True,0.30038536,0.001374066,0.025110465,0.004682911,0.6179109,0.0060193,0.003014118,0.041502878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4595,data/SOD-SemanticDataset/test/COME15K-Easy/COM...,7,vertebrate,4,person,0,covering,3,mater,False,0.11008805,0.0054829586,0.0037603693,0.01637851,0.85486096,0.0021986892,0.003526009,0.0037044452
4596,data/SOD-SemanticDataset/test/COME15K-Easy/COM...,7,vertebrate,7,vertebrate,6,structure,2,domestic_animal,True,0.025481148,0.030964468,0.15987156,0.12275824,0.046617243,0.005959204,0.2390001,0.36934802
4597,data/SOD-SemanticDataset/test/COME15K-Easy/COM...,7,vertebrate,7,vertebrate,3,mater,1,device,True,0.000114533395,0.00047907434,0.00039487163,0.017899925,0.0002784974,0.00034024936,0.0002899217,0.9802029
4598,data/SOD-SemanticDataset/test/COME15K-Easy/COM...,7,vertebrate,7,vertebrate,2,domestic_animal,3,mater,True,0.0053928727,0.0034533,0.04464799,0.024935547,0.009527823,0.0040310402,0.008071735,0.8999396


## 导出完整表格

In [124]:
df.to_csv(model_path + dataset_name +'-测试集预测结果.csv', index=False)